In [ ]:
!pip install nltk


In [ ]:
#importing sentiment Intensity Analizer

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#download vader lexicon
nltk.download('vader_lexicon')
#list of reviews
reviews=[
    "I loved the movie, it was fantastic!",
    "The movie was terrible and boring.",
    "An excellent movie with brilliant acting.",
    "I did not enjoy the movie, it was too slow.",
    "Amazing film! I would watch it again.",
    "Worst movie ever. Do not watch it."
]
#initialize sentiment analyser
sid=SentimentIntensityAnalyzer()
for review in reviews:
  score=sid.polarity_scores(review)
  print(f"{review} and {score}")
  if score['compound']>=0.05:
    print("Positive")
  elif score['compound']<=-0.05:
    print('Negative')
  else:
    print('Neutral')

I loved the movie, it was fantastic! and {'neg': 0.0, 'neu': 0.339, 'pos': 0.661, 'compound': 0.8313}
Positive
The movie was terrible and boring. and {'neg': 0.574, 'neu': 0.426, 'pos': 0.0, 'compound': -0.6597}
Negative
An excellent movie with brilliant acting. and {'neg': 0.0, 'neu': 0.348, 'pos': 0.652, 'compound': 0.8176}
Positive
I did not enjoy the movie, it was too slow. and {'neg': 0.247, 'neu': 0.753, 'pos': 0.0, 'compound': -0.3875}
Negative
Amazing film! I would watch it again. and {'neg': 0.0, 'neu': 0.55, 'pos': 0.45, 'compound': 0.6239}
Positive
Worst movie ever. Do not watch it. and {'neg': 0.406, 'neu': 0.594, 'pos': 0.0, 'compound': -0.6249}
Negative


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [1]:
!pip install nltk scikit-learn pandas numpy tensorflow

  Using cached tensorflow-2.17.0-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.8 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 459.3 kB/s eta 0:00:00
  Using cached tensorflow_intel-2.17.0-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.2/11.0 MB 4.6 MB/s eta 0:00:03
   - -------------------------------------- 0.4/11.0 MB 6.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/11.0 MB 12.4 MB/s eta 0:00:01
   ---------- ----------------------------- 3.0/11.0 MB 17.4 MB/s eta 0:00:01
   -------------------- ------------------- 5.6/11.0 MB 25.5 MB/s eta 0:00:01
   --------------------------- ------------ 7.4/11.0 MB 29.7 MB/s eta 0:00:01

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\Asus\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\tensorflow\\include\\external\\com_github_grpc_grpc\\src\\core\\ext\\filters\\client_channel\\lb_policy\\grpclb\\client_load_reporting_filter.h'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Asus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
# Step 1: Set up Environment
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Step 2: Data Collection
data = pd.read_csv('\content\drive\My Drive\NLP_Projects\Project_SentimentAnalysisonMovieReviews\IMDB_Dataset.csv')

# Step 3: Data Preprocessing
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = nltk.re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = nltk.re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

data['cleaned_review'] = data['review'].apply(preprocess)

# Step 4: Data Manipulation
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['cleaned_review']).toarray()
y = data['sentiment'].map({'positive': 1, 'negative': 0})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Model Training
# Logistic Regression Model
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Alternatively, LSTM Model
max_words = 5000
max_len = 500

lstm_model = Sequential()
lstm_model.add(Embedding(max_words, 128, input_length=max_len))
lstm_model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# For LSTM, you need padded sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_seq = pad_sequences(X_train, maxlen=max_len)
X_test_seq = pad_sequences(X_test, maxlen=max_len)

lstm_model.fit(X_train_seq, y_train, validation_split=0.2, epochs=5, batch_size=64)

# Step 6: Model Testing and Validation
# Evaluate Logistic Regression Model
y_pred_logistic = logistic_model.predict(X_test)
print("Logistic Regression Model")
print("Accuracy:", accuracy_score(y_test, y_pred_logistic))
print(classification_report(y_test, y_pred_logistic))

# Evaluate LSTM Model
loss, accuracy = lstm_model.evaluate(X_test_seq, y_test)
print("LSTM Model")
print("Accuracy:", accuracy)

# Step 7: Prediction
def predict_sentiment(review, model, vectorizer=None, lstm=False):
    review = preprocess(review)
    if lstm:
        review_seq = pad_sequences(vectorizer.transform([review]).toarray(), maxlen=max_len)
        return model.predict(review_seq)[0]
    else:
        review_tfidf = vectorizer.transform([review]).toarray()
        return model.predict(review_tfidf)[0]

new_review = "The movie was great and I enjoyed it."
print("Logistic Regression Sentiment:", predict_sentiment(new_review, logistic_model, vectorizer))
print("LSTM Sentiment:", predict_sentiment(new_review, lstm_model, vectorizer, lstm=True))

# Step 8: End Project


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 23-24: malformed \N character escape (<ipython-input-13-6227422aec0f>, line 20)

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = nltk.re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = nltk.re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

data['cleaned_review'] = data['review'].apply(preprocess)


In [2]:
!pip install keras



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Asus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
